# Depth

By **Franklin Oliveira**

-----

This notebook outputs Depth charts for the `poliqueta` collection. Database is in <font color='blue'>'IBUFRJ27.07.2020 - visualização.xlsx'</font> and <font color='blue'>'MNRJP27.07.2020 - visualização.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

Importing pre-treated data in `1-data_treatment.ipynb`. In this notebook, I'm doing only some minnor adjustments for visualization purposes only. For a full traceback of data treatment, please see the `1-data_treatment` notebook.

In [2]:
# treated_db was the previous db I used to write all this code
NewTable = pd.read_csv('./data/merged_db.csv', sep=';', encoding='utf-8')

In [3]:
NewTable['family'] = NewTable['family'].astype(str)
NewTable['type'] = NewTable['type'].astype(str)

# formatando a string NaN
NewTable['family'] = NewTable['family'].apply(lambda x: 'NaN' if x=='Nan' else x)
NewTable['type'] = NewTable['type'].apply(lambda x: x if str(x) != 'nan' else 'Non-type').astype(str)

<br>

<font size=5>**Color Palette per Order**</font>


<!-- <div class='row' style='padding-top:20px;'>
    <div class='col-md-6'>
        <img src="./src/img1.jpg" width='400px'>
    </div>
    <div class='col-md-6'>
        <img src="./src/img2.jpg" width='400px'>
    </div>
</div>

<br>

A partir das imagens acima, selecionamos cores (centróides) para criar a paleta de cores. Foram elas: 
<ul>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde</li>
    <li style='color:#7A9FBF'><b> #7A9FBF </b># azul</li>
    <li style='color:#D94814'><b> #D94814 </b># laranja</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja 2</li>
    <li style='color:#F2B999'><b> #F2B999 </b># 'cor de pele'</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom 1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom 2</li>
</ul>

A partir das cores "centróides", utilizamos a ferramenta Color Crafter para selecionar diferentes "shades" e auxiliar para categorização em diferentes grupos sugeridos pela equipe de Poliquetas do Museu Nacional. 

<ul>
    <li style='color:#3CA67F'><b> Verde: </b> ['#daffef', '#bbebd3', '#9adabc', '#77c8a5', '#57b791', '#3ca67f', '#2a9670', '#238762', '#257a56']</li>
    <li style='color:#7A9FBF'><b> Azul: </b> ['#e7e5df', '#ccd2d8', '#b2c0d0', '#96afc8', '#7a9fbf', '#5d90b6', '#3c81ae', '#0673a4', '#00669a']</li>
    <li style='color:#D94814'><b> laranja: </b> ['#ffbd84', '#ffaa74', '#ff9760', '#ff814b', '#fc6b36', '#eb5824', '#d94814', '#c83b03', '#b73000']</li>
    <li style='color:#D96236'><b> laranja 2: ['#ffeba9', '#ffd391', '#ffbb7b', '#fda468', '#f18e56', '#e57846', '#d96236', '#cc4d28', '#bf381b']</b> </li>
    <li style='color:#F2B999'><b> cor de pele: ['#ffe9c3', '#fbd0ad', '#f2b999', '#e8a287', '#dd8c76', '#d27666', '#c76158', '#bb4d4b', '#ae393e']</b> </li>
    <li style='color:#A66C4B'><b> marrom 1: ['#d9c6af', '#ccad96', '#c1977c', '#b48061', '#a66c4b', '#975b39', '#874c2c', '#774124', '#683720']</b> </li>
    <li style='color:#732C02'><b> marrom 2: ['#eebd93', '#dfa47a', '#d28d60', '#c37746', '#b4622f', '#a3501d', '#92420e', '#823606', '#732c02']</b> </li>
</ul>



**Colors  (antigas):** 

<ul>
    <li style='color:#41A681'><b> #41A681 </b># verde1</li>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde2</li>
    <li style='color:#7ACAAB'><b> #7ACAAB </b># verde claro</li>
    <li style='color:#78a1a1'><b> #78a1a1 </b># azul</li>
    <li style='color:#8ABFB0'><b> #8ABFB0 </b># azul claro</li>
    <li style='color:#FFB27C'><b> #FFB27C </b># cor de pele clara</li>
    <li style='color:#F29877'><b> #F29877 </b># cor de pele</li>
    <li style='color:#ed845e'><b> #ed845e </b># laranja claro1</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja claro2</li>
    <li style='color:#D95323'><b> #D95323 </b># laranja 1</li>
    <li style='color:#D94B18'><b> #D94B18 </b># laranja 2</li>
    <li style='color:#D9C2AD'><b> #D9C2AD </b># bege</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom claro</li>
    <li style='color:#86471B'><b> #86471B </b># marrom1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom2</li>
    <li style='color:#592202'><b> #592202 </b># marrom escuro1</li>
    <li style='color:#3D1806'><b> #3D1806 </b># marrom escuro2</li>
    <li style='color:#0D0D0D'><b> #0D0D0D </b># preto</li>
</ul>

 -->

In [4]:
# importing customized color palettes
from src.MNViz_colors import *

<br>

---

## Graphs

### Total amount of catalogations per year

x: Start Year (from Start Date)
y: number of catalogations per year

In [5]:
# counting catalog. per year
# teste = NewTable['cataloged_year'].value_counts()
# teste = teste.reset_index().rename(columns={'index':'year', 'cataloged_year':'counts'})

teste = NewTable.groupby(['cataloged_year', 'collection_prefix']).count()
teste = teste['catalog_number'].reset_index().rename(columns={'catalog_number':'counts'})

# min e max para eixo X (year)
min_x = teste['cataloged_year'].min()
max_x = teste['cataloged_year'].max()

In [6]:
temp = alt.Chart(data= teste, width=800, title= 'Number of cataloged specimens per year').mark_bar().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('counts', type='quantitative', title='Counts'), 
    color= alt.Color('collection_prefix', title='Collection')
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/catalogations_per_year.html')
# temp

-----

## Depth per family

In [7]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'collector_full_name', 'country','state','locality', 'type']].copy()

# sorting
teste = teste.sort_values(['min_depth','family'])

# dropping na
teste.dropna(subset=['min_depth'], inplace=True)

# making sure altitude is a floating point number
teste['min_depth'] = teste['min_depth'].astype(float)

# extremes for scale
max_y = teste['min_depth'].max()
min_y = teste['min_depth'].min()

In [8]:
temp = alt.Chart(teste, title='Depth per Family', width=800, height=400).mark_circle().encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y])),
        color= alt.Color('family:N', title='Family', 
                     scale=alt.Scale(domain=list(cores_familia.keys()), 
                                     range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=102)),
    tooltip = alt.Tooltip(['catalog_number', 'order','family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/depth/depth_per_family.html')
# temp

<br>

### Customizing scale for Y axis

as suggested by the Museum team: 200m is a threshold that separates ocean stream.

In [9]:
# escala customizada para o eixo Y
custom_y_scale = []

i = 0

while i < max_y+1:
    if i % 20 == 0 and i <= 200:
        custom_y_scale.append(i)
    elif i % 100 == 0 and i <=500:
        custom_y_scale.append(i)
    elif i % 500 == 0:
        custom_y_scale.append(i)
    i+=1
    
custom_y_scale.append(custom_y_scale[-1] + 500)

# custom_y_scale

In [13]:
data = teste.copy()
data['type'] = data['type'].astype(str)

select_family = alt.selection_multi(fields=['family'], bind='legend')
select_type = alt.selection_multi(fields=['type'], bind='legend')

# background
back = alt.Chart(data, title='Depth per Family', width=800, height=1200).mark_point(filled=True,
                                                                color= 'lightgray', opacity=0.2).encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y]),
              axis= alt.Axis(values=custom_y_scale, tickCount=len(custom_y_scale),
                             labelExpr="datum.value % 50 ? null : datum.label")),
    tooltip = alt.Tooltip(['catalog_number', 'order','family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

# front view
front = alt.Chart(data, title='Depth per Family', width=800, height=1200).mark_point(filled=True).encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y]),
              axis= alt.Axis(values=custom_y_scale, tickCount=len(custom_y_scale),
                             labelExpr="datum.value % 50 ? null : datum.label")),
    color= alt.Color('family', title='Family', 
                     scale=alt.Scale(domain=list(cores_familia.keys()), 
                                     range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=102)),
    shape = alt.Shape('type:N', title='Types', 
                      legend= alt.Legend(columns=4),
                      scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'Non-type'],
                                       range=['triangle', 'square', 'cross','circle'])),
    tooltip = alt.Tooltip(['catalog_number', 'order','family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family).transform_filter(select_type)

g = (back + front)

g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/depth/depth_per_family-custom_scale.html')
# g

<br>

### Interactive version with standard scale

In [14]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'collector_full_name', 'country','state','locality', 'type']].copy()

# sorting
teste = teste.sort_values(['min_depth','family'])

# dropping na
teste.dropna(subset=['min_depth'], inplace=True)

# making sure altitude is a floating point number
teste['min_depth'] = teste['min_depth'].astype(float)

# extremes for scale
max_y = teste['min_depth'].max()
min_y = teste['min_depth'].min()

In [16]:
data = teste.copy()
data['type'] = data['type'].astype(str)

select_family = alt.selection_multi(fields=['family'], bind='legend')
select_type = alt.selection_multi(fields=['type'], bind='legend')

### background
back = alt.Chart(data, title='Depth per Family', width=800, height=400).mark_point(filled=True,
                                                                  color='lightgray', opacity=0.2).encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y])),
    tooltip = alt.Tooltip(['catalog_number', 'order','family','genus','species', 'type',
                           'qualifier', 'start_year','collector_full_name',
                           'country', 'state', 'locality', 'min_depth'])
)

### front view
temp = alt.Chart(data, title='Depth per Family', width=800, height=400).mark_point(filled=True).encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y])),
    color= alt.Color('family', title='Family', 
                     scale=alt.Scale(domain=list(cores_familia.keys()), 
                                     range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=102)),
    shape= alt.Shape('type:N', title='Types', 
                     legend= alt.Legend(columns=4),
                      scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'Non-type'],
                                       range=['triangle', 'square', 'cross','circle'])),
#     opacity= alt.condition(select_type, alt.value(1), alt.value(0)),
    tooltip = alt.Tooltip(['catalog_number', 'order','family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family).transform_filter(select_type)

g = (back + temp)

g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/depth/depth_per_family-shape_types.html')
# g

<br>

## Depth per genus

In [17]:
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species',  'collector_full_name', 'type',
                            'country', 'state', 'locality']].copy()

# making sure altitude is a floating point number
teste['min_depth'] = teste['min_depth'].astype(float)
teste['genus'] = teste['genus'].str.capitalize()

# sorting
teste = teste.sort_values(['min_depth','genus'])

# dropping na
teste = teste.dropna(subset=['min_depth'])

# extremes for y axis
max_y = teste['min_depth'].max()
min_y = teste['min_depth'].min()

In [19]:
select_type = alt.selection_multi(fields= ['type'], bind='legend')
select_family = alt.selection_multi(fields= ['family'], bind='legend')

# filtering depth group
db = teste.sort_values('min_depth', ascending=False)
db['type'] = db['type'].astype(str)  # parsing into string to make selector work for NaN cases

# encoding labels
x_labels = db[['genus', 'min_depth']].groupby('genus').max().reset_index().sort_values('min_depth')['genus'].unique()
y_labels = db['min_depth'].unique()
types = db['type'].unique()
families = db['family'].unique()

# background
back = alt.Chart(db, width= 1500, height=500,
                 title='Depth per Genus').mark_point(filled=True, color='lightgray', opacity=0.2).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[max_y, 0])),
   tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species','type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

# front view
front = alt.Chart(db, width= 1500, height=500,
                 title='Depth per Genus').mark_point(filled=True).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[max_y, 0])),
    color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), 
                                       range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=102)),
    shape= alt.Shape('type:N', title='Types', 
                     legend= alt.Legend(columns=4),
                      scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'Non-type'],
                                       range=['triangle', 'square', 'cross','circle'])),
    tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family).transform_filter(select_type)


g = back + front

g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/depth/genus/depth-per-genus.html')
# g

<br>

## Separating per Min Depth

<font color='red' size=4>Separating groups below and above the 200m sea threshold</font>


In [20]:
genus = teste['genus'].unique()

d = defaultdict()

for gen in genus:
    depth = teste[teste['genus'] == gen]['min_depth'].max()
    d[gen] = depth
        
d = pd.DataFrame(dict(d), index=[0]).transpose().reset_index()
d.columns = ['genus', 'max_depth']

In [21]:
# divisão entre marés (sugerido pelo pessoal do Museu)
threshold = 200

# maior profundidade (>= 500m)
grupo1 = d[d['max_depth'] > threshold]['genus']

# menor profundidade
grupo2 = d[d['max_depth'] <= threshold]['genus']

#### higher depth group

In [23]:
select_type = alt.selection_multi(fields= ['type'], bind='legend')
select_family = alt.selection_multi(fields= ['family'], bind='legend')

# filtering depth group
db = teste[teste['genus'].isin(grupo1)].sort_values('min_depth', ascending=False)
db['type'] = db['type'].astype(str)  # parsing into string to make selector work for NaN cases

# encoding labels
x_labels = db[['genus', 'min_depth']].groupby('genus').max().reset_index().sort_values('min_depth')['genus'].unique()
y_labels = db['min_depth'].unique()
types = db['type'].unique()
families = db['family'].unique()


# background
back = alt.Chart(db, width= 800, height=500,
                 title='Depth per Genus').mark_point(filled=True, color='lightgray', opacity=0.2).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[max_y, 0])),
   tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

# front view
front = alt.Chart(db, width= 800, height=500,
                 title='Depth per Genus').mark_point(filled=True).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[max_y, 0])),
    color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), 
                                       range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=102)),
    shape= alt.Shape('type:N', title='Types', 
                     legend= alt.Legend(columns=4),
                     scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'Non-type'],
                                      range=['triangle', 'square', 'cross','circle'])),
    tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family).transform_filter(select_type)


g = back + front

g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/depth/genus/depth_per_genus-higher-depth.html')
# g

#### lower depth group

In [25]:
select_type = alt.selection_multi(fields= ['type'], bind='legend')
select_family = alt.selection_multi(fields= ['family'], bind='legend')

# filtering depth group
db = teste[teste['genus'].isin(grupo2)].sort_values('min_depth', ascending=False)
db['type'] = db['type'].astype(str)  # parsing into string to make selector work for NaN cases

# encoding labels
x_labels = db[['genus', 'min_depth']].groupby('genus').max().reset_index().sort_values('min_depth')['genus'].unique()
y_labels = db['min_depth'].unique()
types = db['type'].unique()
families = db['family'].unique()


# background
back = alt.Chart(db, width= 800, height=500,
                 title='Depth per Genus').mark_point(filled=True, color='lightgray', opacity=0.2).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[threshold, 0])),
   tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

# front view
front = alt.Chart(db, width= 800, height=500,
                 title='Depth per Genus').mark_point(filled=True).encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending"),
             scale= alt.Scale(domain=x_labels)),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[threshold, 0])),
    color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), 
                                       range=list(cores_familia.values())),
                      legend= alt.Legend(columns=3, symbolLimit=102)),
    shape= alt.Shape('type:N', title='Types', 
                     legend= alt.Legend(columns=4),
                     scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'Non-type'],
                                      range=['triangle', 'square', 'cross','circle'])),
    tooltip = alt.Tooltip(['catalog_number', 'order','family', 'genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family).transform_filter(select_type)


g = back + front

g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/depth/genus/depth_per_genus-lower-depth.html')
# g

<br>

**Thats it!**

-----